In [33]:
with open('tinyshakespeare.txt','r',encoding='utf-8') as f:
    text = f.read()

In [34]:
print('length of dataset in characters:', len(text))


length of dataset in characters: 1115394


In [35]:
#first 100 characters
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [36]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [37]:
#tokenize: convert/map the raw text of the string to some sequence of integers

#encode string -> integer (s to i)
stoi = { ch:i for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] 

#decode integer -> string  (i to s)
itos = { i:ch for i,ch in enumerate(chars)}
decode = lambda i: ''.join([itos[c] for c in i])

print(encode('hii there'))
print(decode(encode('hii there')))



[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [38]:
import tiktoken #BPE, byte pair encoding

enc = tiktoken.get_encoding('gpt2')
enc.n_vocab #instead of 65 tokens they have 50257 tokens, larger vocabulary  but shorter sequence sizes instead of small vocab and large seq. sizes in shakespear

50257

In [39]:
enc.encode('hii there')

[71, 4178, 612]

In [40]:
# encode entire text  and store into a Tensor
import torch
data = torch.tensor(encode(text),dtype=torch.long)
print(data.shape,data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [41]:
#split dataset into train and test/val
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [42]:
#sample chunck size for training (allows for batches of chunks trained parallel), 
# 8 +1 because the 8th predicts the target: the 9th char
block_size = 8
train_data[:block_size+1]


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [43]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [50]:
torch.manual_seed(1337) # we are gonna pull random chunks from different parts of the dataset
batch_size =4 #in parallel process
block_size = 8 #maximum context/time length

def get_batch(split):
    #generate batch
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)

print('targets')
print(yb.shape)
print(yb)

print('---')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b,t]
        print(f'when input is {context.tolist()} the target: {target}')

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
---
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53, 

In [52]:
#simplest neural network for language: BigramLanguageModel

import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337) #reproducability

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits (raw output before activation function) for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward (self, idx, targets):

        #idx and targets are both (Batch, Time) tensors of integers
        logits = self.token_embedding_table(idx)   #(B,T,C) in this case batch = 4, time = 8 and channel is 65

         #negative log-likelyhood loss, a good way to predict the loss
        # we try to call  crossentropy in its functional form, that way we dont have to create a module for it
        # in the documentation for a multi dimensional input the 2nd dimension needs to be the channels so BxCxT
        # we dont want to deal with that so we gonna reshape it to 2dimensions with B*T 

        B,T,C = logits.shape
        logits = logits.view(B*T,C)
        targets = targets.view(B*T)
        loss = F.cross_entropy(logits,targets)

        return logits, loss
    
    def generate(self,idx, max_new_tokens)

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)


torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [60]:
#negative log likelihood 
import math
x = -math.log(1/65) #1/65 probability to pick the right character, we expect the negative log-likelihood to be -ln(1/65) but might vary because of initial random prediction
x

4.174387269895637